In [2]:
import kagglehub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
# Download latest version
path = kagglehub.dataset_download("niharika41298/gym-exercise-data")
file_path = os.path.join(path, "megaGymDataset.csv")  # Replace with the actual filename
df = pd.read_csv(file_path)
print(df.head())

100%|██████████| 120k/120k [00:00<00:00, 39.6MB/s]

Extracting files...
   Unnamed: 0                         Title  \
0           0        Partner plank band row   
1           1  Banded crunch isometric hold   
2           2         FYR Banded Plank Jack   
3           3                 Banded crunch   
4           4                        Crunch   

                                                Desc      Type    BodyPart  \
0  The partner plank band row is an abdominal exe...  Strength  Abdominals   
1  The banded crunch isometric hold is an exercis...  Strength  Abdominals   
2  The banded plank jack is a variation on the pl...  Strength  Abdominals   
3  The banded crunch is an exercise targeting the...  Strength  Abdominals   
4  The crunch is a popular core exercise targetin...  Strength  Abdominals   

  Equipment         Level  Rating RatingDesc  
0     Bands  Intermediate     0.0        NaN  
1     Bands  Intermediate     NaN        NaN  
2     Bands  Intermediate     NaN        NaN  
3     Bands  Intermediate     NaN       

Possible faux dataset for a users input, i assume we have to create a good match for this as well? - Harlee

Great Idea I will be using these initial foundations - Dj

The code below is no longer needed, the generated users are on a seperate csv file.


In [ ]:
from datetime import time
'''
{
    "age": 30,
    "gender": "male",
    "weight": 220,
    "fitness_level": 2, # scale 1-3? intermediate, beginner, expert
    "equipment_owned": ["dumbbell"],
    "lifting_limitations": ["knees"],
    "goals": "build muscle"
}
'''

# possible random profile
num_profiles = 500

equipment = ["Body Only", "Dumbbell", "Barbell", "Other", "Cable"] # in the future, we need datasets with more equipment
limitations = [""] # this needs worked on, im unsure of how we want to
goals = ["Muscle Gain", "Fat Loss", "Strength Building", "Endurance", "Flexibility", "General Fitness", "Core"]

profiles = {}

# make personal details about person
for i in range(num_profiles):
    age = np.random.randint(18, 60)
    gender = np.random.choice(["male", "female"])
    weight = np.random.randint(120, 300)
    fitness_level = np.random.randint(1, 10)
    equipment_owned = np.random.choice(equipment)
    lifting_limitations = np.random.choice(limitations)
    goal = np.random.choice(goals)
    length = np.random.randint(30, 180)

    profiles[i] = (age, gender, weight, fitness_level, equipment_owned, lifting_limitations, goal)
    # put in workouts randomly selected from final_df thats filtered from lifting_restricted, equipment_owned, and goal. In the future we will include optimal for time, fitness level and bio.

TypeError: 'int' object is not callable

Creating a sample of excerises from the full dataset, into a smaller one "final_df"

In [3]:
used_exercises = set()
samples = []
sample_per_combo = 25

goals = df['Type'].unique()
muscles = df['BodyPart'].unique()

for goal in goals:
    for muscle in muscles:
        subset = df[
            (df['Type'] == goal) &
            (df['BodyPart'] == muscle)
        ]

        # Filter out exercises already used
        subset = subset[~subset['Title'].isin(used_exercises)]

        if len(subset) == 0:
            print(f"Skipping {goal} + {muscle} — no unique exercises left")
            continue

        n = min(sample_per_combo, len(subset))
        sampled = subset.sample(n=n, random_state=42)

        # Track used exercises
        used_exercises.update(sampled['Title'])
        samples.append(sampled)

final_df = pd.concat(samples).reset_index(drop=True)
print(f"✅ Final dataset size: {len(final_df)}")

Skipping Plyometrics + Abductors — no unique exercises left
Skipping Plyometrics + Biceps — no unique exercises left
Skipping Plyometrics + Calves — no unique exercises left
Skipping Plyometrics + Forearms — no unique exercises left
Skipping Plyometrics + Glutes — no unique exercises left
Skipping Plyometrics + Lower Back — no unique exercises left
Skipping Plyometrics + Middle Back — no unique exercises left
Skipping Plyometrics + Traps — no unique exercises left
Skipping Plyometrics + Neck — no unique exercises left
Skipping Cardio + Adductors — no unique exercises left
Skipping Cardio + Abductors — no unique exercises left
Skipping Cardio + Biceps — no unique exercises left
Skipping Cardio + Forearms — no unique exercises left
Skipping Cardio + Glutes — no unique exercises left
Skipping Cardio + Lats — no unique exercises left
Skipping Cardio + Lower Back — no unique exercises left
Skipping Cardio + Traps — no unique exercises left
Skipping Cardio + Neck — no unique exercises left
S

Import Alyssa's random user profiles

In [4]:
import pandas as pd

csv_path = "/content/fake_fitness_users_with_tone_restrictions.csv"
user_profiles_df = pd.read_csv(csv_path)

user_profiles_df.head()

,name,age,gender,goal,fitness_level,days_per_week,restrictions
0,Joe Freeman,40,female,gain muscle,beginner,2,none
1,George Evans,22,female,tone,advanced,6,none
2,Lisa Smith,18,female,gain muscle,intermediate,6,none
3,Taylor Owens,22,female,lose fat,advanced,2,none
4,Christine Lopez,21,male,maintain,intermediate,3,none


Beginnings of a rountine generator

In [6]:
import random
import builtins
goal_to_type = {    'Muscle Gain': 'Strength',
        'muscle gain': 'Strength',
        'gain muscle': 'Strength',
        'fat loss': 'Cardio',
        'lose fat': 'Cardio',
        'strength building': 'Powerlifting',
        'endurance': 'Cardio',
        'flexibility': 'Stretching',
        'general fitness': ['Strength', 'Cardio'],
        'core': ['Strength', 'Cardio', 'Plyometrics'],
        'tone': ['Strength', 'Cardio'],
        'maintain': ['Strength', 'Cardio']
}
goal_durations = {
    "gain muscle": (45, 75),
    "lose fat": (30, 120),
    "tone": (30, 60),
    "maintain": (30, 60),
    "strength": (60, 90),
    "endurance": (60, 120),
    "flexibility": (20, 60),
    "core": (15, 60)
}
def map_goal_to_type(goal): # Im thinking that the bug causing very strength focused workouts may be here
    goal = goal.strip().lower()

    for key in goal_to_type:
        key_normalized = key.lower()
        if goal in key_normalized or key_normalized in goal:
            mapped = goal_to_type[key]
            return random.choice(mapped) if isinstance(mapped, list) else mapped

    # If no match found, fall back
    return 'Strength'

def divide_duration(duration, num_exercises, min_exercise=2, max_exercise=25):
    range = builtins.range # a bug that set me back my an hour was that somewhere within the code "range" got changed to an int.
    total_units = duration
    min_units = min_exercise
    max_units = max_exercise

    durations = [min_units] * num_exercises
    remaining_units = total_units - (min_units * num_exercises)

    while remaining_units > 0:
        for i in range(num_exercises):
            if durations[i] < max_units and remaining_units > 0:
                durations[i] += 1
                remaining_units -= 1
    return durations

def generate_user_routine(user_row, exercises_df, num_exercises=5):

    goal = user_row['goal'].strip().lower()
    restriction = user_row['restrictions'].lower()
    workout_type = map_goal_to_type(goal)
    # Filter exercises by selected type
    filtered = exercises_df[exercises_df['Type'] == workout_type]

    # Remove exercises that might conflict with restriction
    if restriction != 'none':
        filtered = filtered[~filtered['BodyPart'].str.lower().str.contains(restriction)]

    # Pick different exercises for each user
    if len(filtered) < num_exercises:
        exercises = filtered['Title'].tolist()
    else:
        exercises = filtered.sample(n=num_exercises)['Title'].tolist()

    low, high = goal_durations.get(goal.strip().lower(), (30, 60))
    duration = random.randrange(low, high + 1, 5)
    durations = divide_duration(duration, len(exercises))

    exercises_with_time = [
        {'exercise': name, 'duration_minutes': time}
        for name, time in zip(exercises, durations)
    ]
    return {
        'name': user_row['name'],
        'goal': goal,
        'workout_type': workout_type,
        'exercises': exercises_with_time,
        'duration': duration
    }

# Apply to first 5 users again
user_routines = [generate_user_routine(row, final_df) for _, row in user_profiles_df.head(500).iterrows()]
for routine in user_routines:
    goal = routine['goal']
    restriction = routine.get('restriction', 'none').lower()
    exercises = routine['exercises']



In [12]:
import tensorflow as tf
import pandas as pd
restriction_vocabulary = [
    'none', 'neck', 'left_shoulder', 'right_shoulder', 'left_wrist', 'right_wrist',
    'left_ankle', 'right_ankle', 'left_leg', 'right_leg',
    'left_foot', 'right_foot', 'upper_back', 'lower_back'
]


useable_data = []
for routine in user_routines: # turn our data into readable data by the neural network
    goal = routine['goal']
    name = routine['name']

    exercises = [excercise['exercise'].lower() for excercise in routine['exercises']]
    duration = routine['duration']
    restriction = user_profiles_df[user_profiles_df['name'] == name]['restrictions'].values[0].lower()

    useable_data.append({
        'name': name,
        'goal': goal,
        'restriction': restriction,
        'duration': duration,
        'exercises': exercises
    })
'''
    for ex in exercises:
        useable_data.append({
            'goal': goal.lower(),
            'restriction': restriction,
            'exercise': ex['exercise'].lower(),
            'duration': ex['duration_minutes']
        })

'''
df = pd.DataFrame(useable_data)
print(df.head())


#this took some research on how we need to format our goals and restrictions to exercises that the neural network can understand
#we have to have it grab the string values from goal, restriction, and exercise.
#we use one_hot because they are not ordered in any way
#exercises are multi_hot because of the neural network being a multi-label classifier and picking from them
goal_lookup = tf.keras.layers.StringLookup(output_mode='one_hot')
restriction_lookup = tf.keras.layers.StringLookup(vocabulary=restriction_vocabulary, output_mode='multi_hot')
exercise_lookup = tf.keras.layers.StringLookup(output_mode='multi_hot')
# take in each unique feature for goal, restriction, and exercises
goal_lookup.adapt(df['goal'])
all_exercises = [ex for sublist in df['exercises'] for ex in sublist] # loop through each profile
exercise_lookup.adapt(all_exercises)

# finally encodes it
goal_encoded = goal_lookup(df['goal'].values)
restriction_lists = df['restriction'].str.lower().str.split(', ')  # list of lists, also filters
restriction_encoded = tf.stack([
    restriction_lookup(tf.constant(restrictions)) for restrictions in restriction_lists
])
exercise_encoded = tf.stack([
    exercise_lookup(ex_list) for ex_list in df['exercises']
])
#verification
# first number should be number of profiles, second should be the amount per category
print("Goal encoded shape:", goal_encoded.shape)
print("Restriction encoded shape:", restriction_encoded.shape)
print("Exercise encoded shape:", exercise_encoded.shape)


#before duration can effectively be used we need actual data or to find a way to get filtered data.
#currently, the neural network will use duration to pick exercises that people with similar amount of times have.
#unfortuntely we did not have enough time to filter through the exercises and group by average time durations.
duration_scaled = df['duration'].values / df['duration'].max() # changes the durations to a value between 0-1
duration_scaled = tf.convert_to_tensor(duration_scaled, dtype=tf.float32) #convert to a format readable by the neural network
duration_scaled = tf.expand_dims(duration_scaled, axis=1) # nn expects a 2d input
#combine it
inputs = tf.concat([
    tf.cast(goal_encoded, tf.float32),
    tf.cast(restriction_encoded, tf.float32),
    duration_scaled
], axis=1)
#input_size is the total size of unique goals, restrictions, and duration
input_size = goal_encoded.shape[1] + restriction_encoded.shape[1] + 1 #+1 for duration

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(input_size,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(exercise_lookup.vocabulary_size(), activation='sigmoid')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(inputs, exercise_encoded, epochs=30, batch_size=32, validation_split=0.1)
model.save("workout_recommender_model.keras")

#Notes
#the number of exercises may be limited in this initial model due to the sample size not containing all exercises
#for the future we need to gather more refined data from maybe another neural network or real data.
#we also need to implement a negative or positive association with the exercise list
#I am also unsure of why accuracy has those values during the 3rd and 4th epoch

              name         goal restriction  duration  \
0      Joe Freeman  gain muscle        none        60   
1     George Evans         tone        none        30   
2       Lisa Smith  gain muscle        none        70   
3     Taylor Owens     lose fat        none        35   
4  Christine Lopez     maintain        none        50   

                                           exercises  
0  [lying face down plate neck resistance, machin...  
1  [paul carter romanian deadlift, traveling thig...  
2  [seated one-arm dumbbell palms-down wrist curl...  
3  [sled push-, burpee, jumping jack, high knees,...  
4  [hm running lunge, defensive slide, football u...  
Goal encoded shape: (500, 5)
Restriction encoded shape: (500, 15)
Exercise encoded shape: (500, 392)
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.0024 - loss: 29.7961 - val_accuracy: 0.0000e+00 - val_loss: 29.6165
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.0113 - loss: 29.4464 - val_accuracy: 0.0000e+00 - val_loss: 29.2039
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 7.3029e-04 - loss: 28.7554 - val_accuracy: 0.0000e+00 - val_loss: 28.2685
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 7.3029e-04 - loss: 27.4070 - val_accuracy: 0.0000e+00 - val_loss: 27.7758
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0026 - loss: 26.9617 - val_accuracy: 0.0400 - val_loss: 32.6230
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0166 - loss: 32.4900 - val_accuracy: 0.0400 - val_loss: 45.5777
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0491 - loss: 45.4730 - val_accuracy: 0.0800 - val_loss: 69.4672
Epoch 8/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1031 - loss: 66.0212 - 

In [22]:
# because accuracy isnt the best measure of how the model is doing lets have it generate some exercises
def recommend_exercises(goal, restriction, duration, exercise_count=5):
    goal_input = goal_lookup([goal.lower()])
    restrictions = [r.strip() for r in restriction.lower().split(',')] if restriction.lower() != 'none' else ['none']
    restriction_input = tf.reduce_sum(
        tf.one_hot(restriction_lookup(restrictions), depth=restriction_lookup.vocabulary_size()),
        axis=0, keepdims=True
    )
    duration_scaled = duration / df['duration'].max()
    duration_input = tf.convert_to_tensor([[duration_scaled]], dtype=tf.float32)

    input = tf.concat([
        tf.cast(goal_input, tf.float32),
        tf.cast(restriction_input, tf.float32),
        duration_input
    ], axis=1)

    predictions = model.predict(input)[0]
    top_exercises = tf.argsort(predictions, direction='DESCENDING')[:exercise_count].numpy()
    exercise_names = exercise_lookup.get_vocabulary()

    return [exercise_names[i] for i in top_exercises]

print(recommend_exercises("flexiblity", "lower_back", 45))



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
[np.str_('fyr sprinter burpee'), np.str_('sled push-'), np.str_('plate push'), np.str_('recumbent bike'), np.str_('defensive slide')]
